In [46]:
import pandas as pd
from backtesting import Strategy
from backtesting.lib import crossover, TrailingStrategy
from backtesting import Backtest
from backtesting.test import SMA

import numpy as np

data = pd.read_csv("BTC-USD.csv", parse_dates=[0], date_parser=pd.to_datetime, index_col = [0], header = 0)

# data = pd.read_csv("BTC-USD-100day.csv", parse_dates=[0], date_parser=pd.to_datetime, index_col = [0], header = 0)


In [47]:
# def SMA(values, n):
#     """
#     Return simple moving average of `values`, at
#     each step taking into account `n` previous values.
#     """
#     return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 8
    sltr=500e-4

    def init(self):
        # Precompute the two moving averages
        super().init()
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
    
    def next(self):

        super().next()

        sltr = self.sltr
        if len(self.trades) > 0:
            print("Active Trade: " + str(len(self.trades)))

        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.High[-1] * (1-sltr))
                print("entry_time: " + str(trade.entry_time) + " ,sl: " + str(trade.sl))
            # else:
            #     trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if (self.data.Open[-1] > self.sma1[-2] and 
        self.sma1[-2] > self.sma2[-2] and
        self.data.Open[-1] > self.data.Low[-2] and 
        self.data.Low[-2] > self.data.Low[-3] and
        self.data.Low[-3] > self.data.Low[-4]):
            print ("Open[-1]: " + str(self.data.Open[-1]) +
            " Low[-2]: " + str( self.data.Low[-2]) +
            " Low[-3]: " + str( self.data.Low[-3]) +
            " Low[-4]: " + str( self.data.Low[-4]) +
            " sma1: " + str(self.sma1[-2]) +
            " sma2: " + str(self.sma2[-2])
            )
            self.buy(limit=self.data.Open[-1], sl=self.data.Low[-2])


In [48]:
bt = Backtest(data, SmaCross, cash=1000000, commission=.002)
stats = bt.run()
stats
# stats['_equity_curve']


Open[-1]: 11755.5 Low[-2]: 11475.299805 Low[-3]: 10989.200195 Low[-4]: 10470.299805 sma1: 11403.440234599999 sma2: 11302.1376955
Active Trade: 1
entry_time: 2018-01-30 00:00:00 ,sl: 11475.299805
Open[-1]: 8616.129883 Low[-2]: 8295.469727 Low[-3]: 7884.709961 Low[-4]: 7637.859863 sma1: 8199.9541018 sma2: 8175.870056375
Active Trade: 1
entry_time: 2018-02-12 00:00:00 ,sl: 8536.6239259
Open[-1]: 9488.320313 Low[-2]: 8599.919922 Low[-3]: 8455.410156 Low[-4]: 8141.430176 sma1: 8754.2760744 sma2: 8549.40631125
Open[-1]: 10135.700195 Low[-2]: 9395.580078 Low[-3]: 8599.919922 Low[-4]: 8455.410156 sma1: 9063.1760744 sma2: 8867.543884499999
Open[-1]: 10207.5 Low[-2]: 9824.820313 Low[-3]: 9395.580078 Low[-4]: 8599.919922 sma1: 9483.9621096 sma2: 9113.582702874999
Open[-1]: 11123.400391 Low[-2]: 10149.400391 Low[-3]: 9824.820313 Low[-4]: 9395.580078 sma1: 9921.188085999998 sma2: 9410.547668625
Active Trade: 1
entry_time: 2018-02-19 00:00:00 ,sl: 10710.10981475
Open[-1]: 10552.599609 Low[-2]: 10326

c:\Users\Tommy Yung\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2018-02-12 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\Tommy Yung\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\backtesting.py:912: UserWarning: (2018-03-06 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/k

Start                     2018-01-01 00:00:00
End                       2022-11-17 00:00:00
Duration                   1781 days 00:00:00
Exposure Time [%]                   35.016835
Equity Final [$]               2339169.383912
Equity Peak [$]                3962005.159543
Return [%]                         133.916938
Buy & Hold Return [%]               21.742974
Return (Ann.) [%]                   19.012736
Volatility (Ann.) [%]               51.854002
Sharpe Ratio                         0.366659
Sortino Ratio                        0.763888
Calmar Ratio                         0.450445
Max. Drawdown [%]                  -42.208745
Avg. Drawdown [%]                  -10.360478
Max. Drawdown Duration      594 days 00:00:00
Avg. Drawdown Duration       83 days 00:00:00
# Trades                                  237
Win Rate [%]                        37.552743
Best Trade [%]                      33.154823
Worst Trade [%]                    -10.779139
Avg. Trade [%]                    

In [49]:
bt.plot()

Row(id='11007', ...)